## Load all libraries

In [36]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss

## Load train set and build simple features

In [32]:
%run "scripts/start.py"
df_full, df_train, df_test = IO_full()
print(len(df_train))
df_full, df_test = None, None # This notebook is for validation, no test set needed
df_train = make_simple_features(df_train)
simple_feats = ["bathrooms", "bedrooms", "price",        
                "num_photos", "num_features", "num_description_words",                    
                "created_month", "created_day", "created_hour"
            ]

49352


## Load features

In [20]:
cluster_feat = pd.read_csv("features/mk/train_kmean_ft.csv")
assert len(cluster_feat) == 49352

## Merge all features with df_train

In [33]:
df_train = pd.merge(df_train, cluster_feat, left_index=True, right_index=True)

## Fit model 

In [38]:
def fit_classifier(X_train, y_train):
    from sklearn.ensemble import RandomForestClassifier as RFC
    clf=RFC(n_estimators=1000, random_state=42)
    clf.fit(X_train, y_train)
    return clf   

features = simple_feats + ["cluster_35"]

target_num_map={"high":0, "medium":1, "low":2}
df_train["interest_level_num"]=df_train["interest_level"].apply(lambda x: target_num_map[x]).values

train_index, valid_index = IO_SkF()

print("Referring to both the seen data and the train fold as 'train' is confusing so tell me if you have better terminology")
X_train, y_train =df_train.loc[train_index, features].copy(deep=True), df_train.loc[train_index, "interest_level_num"].copy(deep=True) 
X_val, y_val = df_train.loc[valid_index, features].copy(deep=True), df_train.loc[valid_index, "interest_level_num"].copy(deep=True)

for col in features:
    assert len(X_train[X_train[col].isnull()]) == 0   

clf = fit_classifier(X_train, y_train)
y_val_pred = clf.predict_proba(X_val)
print(log_loss(y_val, y_val_pred))

Referring to both the seen data and the train fold as 'train' is confusing so tell me if you have better terminology
0.641364199123
